In [1]:
# faire du auguementation de data (changement de lumi...)
#faceapi framework python
#faire de la doc
# teste
#mlops (re entrainé son ia)
# cold storage / hot storage / data warehouse 
#bucket google , s3 amazon

import urllib.request
import zipfile
import io

# URL du fichier zip contenant les images
url = "https://github.com/Fred-shenron/ia_img/raw/main/train.zip"

# # Télécharger le fichier zip et extraire les images
with urllib.request.urlopen(url) as url_response:
    with zipfile.ZipFile(io.BytesIO(url_response.read())) as zip_file:
      zip_file.extractall()
      print("Images extraites avec succès.")

urlTest = "https://github.com/Fred-shenron/ia_img/raw/main/test.zip"

# Télécharger le fichier zip et extraire les images
with urllib.request.urlopen(urlTest) as url_response:
    with zipfile.ZipFile(io.BytesIO(url_response.read())) as zip_file:
        zip_file.extractall()
        print("Images extraites avec succès.")

urlVal= "https://github.com/Fred-shenron/ia_img/raw/main/val.zip"

# Télécharger le fichier zip et extraire les images
with urllib.request.urlopen(urlVal) as url_response:
    with zipfile.ZipFile(io.BytesIO(url_response.read())) as zip_file:
        zip_file.extractall()
        print("Images extraites avec succès.")


Images extraites avec succès.
Images extraites avec succès.
Images extraites avec succès.


In [2]:
import urllib.request
import zipfile
import io
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping

# Votre code d'importation des images ici

# Chemins vers les dossiers d'images
train_dir = "train"
test_dir = "test"
val_dir = "val"

# Prétraitement des images
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Générateurs d'images
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=32, class_mode='categorical',)
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(150, 150), batch_size=32, class_mode="categorical")
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(150, 150), batch_size=32, class_mode="categorical")


Found 7618 images belonging to 3 classes.
Found 788 images belonging to 3 classes.
Found 26 images belonging to 3 classes.


In [3]:
# Construction du modèle CNN
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation="relu"),
    Dropout(0.5),
    Dense(3, activation="softmax")  # Modifiez cette ligne
])

# Compilation du modèle
model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])  # Modifiez cette ligne

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
# Entraînement du modèle
history = model.fit(train_generator, epochs=20, validation_data=val_generator, callbacks=[early_stopping])

# Évaluation du modèle
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy}")

Epoch 1/20
239/239 [==============================] - 83s 300ms/step - loss: 0.2495 - accuracy: 0.9153 - val_loss: 0.2293 - val_accuracy: 0.8846
Epoch 2/20
239/239 [==============================] - 70s 293ms/step - loss: 0.0933 - accuracy: 0.9664 - val_loss: 0.1284 - val_accuracy: 0.9615
Epoch 3/20
239/239 [==============================] - 71s 295ms/step - loss: 0.0893 - accuracy: 0.9664 - val_loss: 0.1526 - val_accuracy: 0.9615
Epoch 4/20
239/239 [==============================] - 71s 296ms/step - loss: 0.0693 - accuracy: 0.9768 - val_loss: 0.3592 - val_accuracy: 0.8077
Epoch 5/20
239/239 [==============================] - 71s 299ms/step - loss: 0.0403 - accuracy: 0.9857 - val_loss: 0.0680 - val_accuracy: 0.9615
Epoch 6/20
239/239 [==============================] - 70s 292ms/step - loss: 0.0332 - accuracy: 0.9871 - val_loss: 0.1208 - val_accuracy: 0.9231
Epoch 7/20
239/239 [==============================] - 70s 294ms/step - loss: 0.0272 - accuracy: 0.9909 - val_loss: 0.0711 - val_ac

In [ ]:
# Afficher les images avec leurs prédictions et leurs labels

import math

def display_image_predictions(images, true_labels, predictions):
    num_images = len(images)
    grid_size = math.ceil(math.sqrt(num_images))
    
    fig, axes = plt.subplots(grid_size, grid_size, figsize=(15, 15))
    fig.subplots_adjust(hspace=0.5, wspace=0.5)

    for i, (img, true_label, pred) in enumerate(zip(images, true_labels, predictions)):
        row, col = divmod(i, grid_size)
        axes[row, col].imshow(img)
        axes[row, col].set_title(f"Vraie catégorie : {true_label}\nPrédiction : {pred}")
        axes[row, col].axis('off')

    # Masquer les axes inutilisés
    for j in range(i + 1, grid_size * grid_size):
        row, col = divmod(j, grid_size)
        axes[row, col].axis('off')

    plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Évaluer le modèle sur l'ensemble de test
test_loss, test_acc = model.evaluate(test_generator)
print('Accuracy on test set:', test_acc)

# Créer un graphique pour afficher les résultats
fig, ax = plt.subplots()
ax.plot([0, 1], [0, 1], 'k--')
ax.plot(test_loss, test_acc, 'bo', label='Test')
ax.set_xlabel('Loss')
ax.set_ylabel('Accuracy')
ax.set_title('Accuracy on test set: {:.2f}'.format(test_acc))
ax.legend()
plt.show()

In [ ]:
#__________________________ EXPORT _____________________________________

In [ ]:
pip install tensorflowjs

In [ ]:
model.save('my_model.h5')


In [ ]:
import zipfile

def compress_model(model_path, zip_path):
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        zipf.write(model_path, arcname=model_path.split('/')[-1])

model_path = 'my_model.h5'
zip_path = 'my_model.zip'

compress_model(model_path, zip_path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Définissez le chemin d'accès dans Google Drive où vous souhaitez enregistrer le modèle
save_path = "/content/drive/MyDrive/my_model.h5"

# Enregistrez le modèle
model.save(save_path)